# Countries CO2 Emission and Energy Consumption/Production

This data is a collection of some big factors that play into C02 Emissions, with everything from the Production and Consumption of each type of major energy source for each country and its pollution rating each year. https://www.kaggle.com/datasets/lobosi/c02-emission-by-countrys-grouth-and-population 

In [4]:
import pandas as pd
import json
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
from azure.storage.blob import BlobClient

from config import account
from config import container
from config import credential

blob = BlobClient(account_url=f"https://{account}.blob.core.windows.net",
                  container_name=container,
                  blob_name="energy.csv",
                  credential=credential)


with open("energy.csv", "wb") as f:
    data = blob.download_blob()
    data.readinto(f)

## CLEANING DATA

In [6]:
# Load the data into a pandas DataFrame
energy = pd.read_csv('energy.csv')


### Exploratory analysis  of dataset 

In [7]:
energy.head()

,Unnamed: 0,Country,Energy_type,Year,Energy_consumption,Energy_production,GDP,Population,Energy_intensity_per_capita,Energy_intensity_by_GDP,CO2_emission
0,0,World,all_energy_types,1980,292.899790,296.337228,27770.910281,4.298127e+06,68.145921,10.547,4946.627130
1,1,World,coal,1980,78.656134,80.114194,27770.910281,4.298127e+06,68.145921,10.547,1409.790188
2,2,World,natural_gas,1980,53.865223,54.761046,27770.910281,4.298127e+06,68.145921,10.547,1081.593377
3,3,World,petroleum_n_other_liquids,1980,132.064019,133.111109,27770.910281,4.298127e+06,68.145921,10.547,2455.243565
4,4,World,nuclear,1980,7.575700,7.575700,27770.910281,4.298127e+06,68.145921,10.547,0.000000


In [8]:
energy.shape

(55440, 11)

Check datatypes and overview of nulls 

In [9]:
energy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55440 entries, 0 to 55439
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   55440 non-null  int64  
 1   Country                      55440 non-null  object 
 2   Energy_type                  55440 non-null  object 
 3   Year                         55440 non-null  int64  
 4   Energy_consumption           44287 non-null  float64
 5   Energy_production            44289 non-null  float64
 6   GDP                          40026 non-null  float64
 7   Population                   46014 non-null  float64
 8   Energy_intensity_per_capita  50358 non-null  float64
 9   Energy_intensity_by_GDP      50358 non-null  float64
 10  CO2_emission                 51614 non-null  float64
dtypes: float64(7), int64(2), object(2)
memory usage: 4.7+ MB


In [10]:
energy.describe()

,Unnamed: 0,Year,Energy_consumption,Energy_production,GDP,Population,Energy_intensity_per_capita,Energy_intensity_by_GDP,CO2_emission
count,55440.000000,55440.0000,44287.000000,4.428900e+04,40026.000000,4.601400e+04,50358.000000,50358.000000,51614.000000
mean,27719.500000,1999.5000,1.537811,1.532700e+00,827.144126,6.263020e+04,71.898914,3.695104,78.800082
std,16004.293799,11.5435,15.456596,1.530356e+01,5981.703144,4.562088e+05,113.728738,4.590735,902.221463
min,0.000000,1980.0000,-0.163438,-1.000000e-39,0.124958,1.147100e+01,0.000000,0.000000,-0.005130
25%,13859.750000,1989.7500,0.000000,0.000000e+00,9.737780,1.141950e+03,3.799939,0.899446,0.000000
50%,27719.500000,1999.5000,0.018381,5.121971e-04,47.757100,6.157680e+03,29.779260,2.987593,0.000000
75%,41579.250000,2009.2500,0.209422,1.125410e-01,263.687100,2.004290e+04,95.523627,4.969454,4.318822
max,55439.000000,2019.0000,601.040490,6.115090e+02,127690.247059,7.714631e+06,1139.320598,166.913605,35584.933498


Check for duplicates 

In [11]:
energy.duplicated().sum()

0

Drop extra index column Unnamed:0

In [12]:
energy = energy.drop('Unnamed: 0', axis=1)

Explore Countries 

In [13]:
energy['Country'].unique()

array(['World', 'Afghanistan', 'Albania', 'Algeria', 'American Samoa',
       'Angola', 'Antarctica', 'Antigua and Barbuda', 'Argentina',
       'Armenia', 'Aruba', 'Australia', 'Austria', 'Azerbaijan',
       'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium',
       'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia',
       'Bosnia and Herzegovina', 'Botswana', 'Brazil',
       'British Virgin Islands', 'Brunei', 'Bulgaria', 'Burkina Faso',
       'Burma', 'Burundi', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada',
       'Cayman Islands', 'Central African Republic', 'Chad', 'Chile',
       'China', 'Colombia', 'Comoros', 'Congo-Brazzaville',
       'Congo-Kinshasa', 'Cook Islands', 'Costa Rica', 'Croatia', 'Cuba',
       'Cyprus', 'Czech Republic', 'Côte d’Ivoire', 'Denmark', 'Djibouti',
       'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt',
       'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia',
       'Eswatini', 'Ethiopia', 'Falkland Islands', 'Faroe Islands',

drilling in to the data for former countries and countries where the region gets broken down.  

In [14]:
energy[energy['Country'] == 'Former Yugoslavia']
energy[energy['Country'] == 'Former U.S.S.R.']
energy[energy['Country'] == 'Former Serbia and Montenegro']
energy[energy['Country'] == 'Germany, West']



,Country,Energy_type,Year,Energy_consumption,Energy_production,GDP,Population,Energy_intensity_per_capita,Energy_intensity_by_GDP,CO2_emission
486,"Germany, West",all_energy_types,1980,11.431371,4.650354,NaN,NaN,0.0,0.0,NaN
487,"Germany, West",coal,1980,3.184389,3.150368,NaN,NaN,0.0,0.0,NaN
488,"Germany, West",natural_gas,1980,1.889664,0.602952,NaN,NaN,0.0,0.0,NaN
489,"Germany, West",petroleum_n_other_liquids,1980,5.722675,0.199570,NaN,NaN,0.0,0.0,NaN
490,"Germany, West",nuclear,1980,0.465915,0.465915,NaN,NaN,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
54541,"Germany, West",coal,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54542,"Germany, West",natural_gas,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54543,"Germany, West",petroleum_n_other_liquids,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54544,"Germany, West",nuclear,2019,NaN,NaN,NaN,NaN,NaN,NaN,0.0


Drop Countries that do not exist anymore

In [15]:

dropCountry= ['Former Czechoslovakia', 'Former Serbia and Montenegro','Former U.S.S.R.', 'Former Yugoslavia', 'Germany, East',
       'Germany, West', 'Hungary', 'Poland']

for country in dropCountry:
    value = energy[energy['Country']==country].index
    energy.drop(labels=value, axis=0, inplace=True)

Drop rows with 7 or less non NaN values (or 3 or more NaN values). This s to elimante countries that did not exist during certain years. 


In [16]:
energy.dropna(thresh=7, axis=0, inplace=True)

Check missing values

In [17]:
energy.isnull().sum()

Country                            0
Energy_type                        0
Year                               0
Energy_consumption              6292
Energy_production               6291
GDP                            10202
Population                      4310
Energy_intensity_per_capita      984
Energy_intensity_by_GDP          984
CO2_emission                    1217
dtype: int64

We need CO2 emissions, therefore dropped nulls under CO2_emission column. 


In [18]:
energy.dropna(subset=['CO2_emission'], inplace=True)

All nulls for Energy consumption and production come from Nuclear. Not that many countries provide nor have nuclear as an energy source.

In [19]:
energy[energy['Energy_type']== 'nuclear'].isnull().sum()

Country                           0
Energy_type                       0
Year                              0
Energy_consumption             6291
Energy_production              6291
GDP                            1074
Population                      116
Energy_intensity_per_capita       0
Energy_intensity_by_GDP           0
CO2_emission                      0
dtype: int64

Split nuclear energy type from the rest of the dataset to replace Nan with 0

In [20]:
nuclear= energy[energy['Energy_type']== 'nuclear']
#Replace NaN to 0
nuclear[['Energy_consumption']] = nuclear[['Energy_consumption']].fillna(0)
nuclear[['Energy_production']] = nuclear[['Energy_production']].fillna(0)
#The rest of the main dataset
energy2= energy[energy['Energy_type'] != 'nuclear']
#Combine the split datasets
energy= pd.concat([nuclear, energy2]).sort_index()

C:\Users\zach3\AppData\Local\Temp\ipykernel_27292\1422855934.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nuclear[['Energy_consumption']] = nuclear[['Energy_consumption']].fillna(0)
C:\Users\zach3\AppData\Local\Temp\ipykernel_27292\1422855934.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nuclear[['Energy_production']] = nuclear[['Energy_production']].fillna(0)


In [21]:
energy['Energy_type'].value_counts()
energy['Country'].value_counts()

World                  240
United States          240
North Korea            236
Morocco                236
Mozambique             236
                      ... 
Montenegro              80
Serbia                  80
Kosovo                  68
South Sudan             44
Hawaiian Trade Zone     31
Name: Country, Length: 223, dtype: int64

Group nulls under GDP & Country to find which countries have more than 90 GDP values. Considering we have 236 entries per country, 90 is close to a 40% cutoff.


In [22]:
gdpnull = energy['GDP'].isnull().groupby(energy['Country']).sum()
gdp_nulls= pd.DataFrame(gdpnull).reset_index()

In [23]:
GDP90 = gdp_nulls[gdp_nulls['GDP']>=90]
GDP90

,Country,GDP
3,American Samoa,134
5,Antarctica,196
45,Cook Islands,196
64,Falkland Islands,196
65,Faroe Islands,183
69,French Guiana,196
70,French Polynesia,196
76,Gibraltar,196
81,Guam,134
126,Micronesia,161


Dropping countries with too many GDP nulls

In [24]:
gdplist= [x for x in GDP90['Country']]

for country in gdplist:
    value = energy[energy['Country']==country].index
    energy.drop(labels=value, axis=0, inplace=True)

Checking the count of attributes per year by grouping year with corresponding columns.

In [25]:
energy.groupby('Year')['Energy_consumption', 'Energy_production', 'GDP', 'Population', 'Energy_intensity_per_capita', 'Energy_intensity_by_GDP', 'CO2_emission'].count()

C:\Users\zach3\AppData\Local\Temp\ipykernel_27292\2958038831.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  energy.groupby('Year')['Energy_consumption', 'Energy_production', 'GDP', 'Population', 'Energy_intensity_per_capita', 'Energy_intensity_by_GDP', 'CO2_emission'].count()


,Energy_consumption,Energy_production,GDP,Population,Energy_intensity_per_capita,Energy_intensity_by_GDP,CO2_emission
Year,,,,,,,
1980,336,336,109,331,331,331,336
1981,1003,1003,310,978,979,979,1003
1982,1003,1003,310,978,979,979,1003
1983,1003,1003,310,978,979,979,1003
1984,1003,1003,310,978,979,979,1003
1985,1003,1003,310,978,979,979,1003
1986,1005,1005,310,980,981,981,1005
1987,1004,1004,310,984,980,980,1004
1988,1008,1008,972,1008,984,984,1008


GDP is low for countries between 1980 and 1987. 1980 in particular has low energy consumption and production. Excluding years before 1988 for ease of use


In [26]:
energy= energy[energy['Year']>= 1988]

In [27]:
energy.head()

,Country,Energy_type,Year,Energy_consumption,Energy_production,GDP,Population,Energy_intensity_per_capita,Energy_intensity_by_GDP,CO2_emission
11088,World,all_energy_types,1988,345.560876,347.412863,42106.595403,4.927545e+06,70.128405,8.206811,21163.840556
11089,World,coal,1988,96.873178,98.484482,42106.595403,4.927545e+06,70.128405,8.206811,8930.924825
11090,World,natural_gas,1988,71.010048,71.852938,42106.595403,4.927545e+06,70.128405,8.206811,3571.676242
11091,World,petroleum_n_other_liquids,1988,133.445814,132.485303,42106.595403,4.927545e+06,70.128405,8.206811,8661.239490
11092,World,nuclear,1988,19.226897,19.226897,42106.595403,4.927545e+06,70.128405,8.206811,0.000000


In [28]:
energy['Energy_type'].value_counts()
energy['Country'].value_counts()

World          192
Nepal          192
New Zealand    192
Nicaragua      192
Niger          192
              ... 
Timor-Leste     98
Montenegro      80
Serbia          80
Kosovo          68
South Sudan     44
Name: Country, Length: 199, dtype: int64

Removing rows with missing energy intensity by GDP missing values 

In [29]:
energy['Energy_intensity_by_GDP'].isnull().sum()

795

In [30]:
eip= pd.DataFrame(energy['Energy_intensity_by_GDP'].isnull().groupby(energy['Country']).sum()).reset_index()
#Countries with missing values
eip= eip[eip['Energy_intensity_by_GDP'] > 0]
eip

,Country,Energy_intensity_by_GDP
70,Greenland,155
80,Iceland,160
124,Netherlands Antilles,160
180,Trinidad and Tobago,160
184,U.S. Virgin Islands,160


In [31]:
#Making a list of the countries 
eiplist= [x for x in eip['Country']]
#Dropping countries from eiplist
for country in eiplist:
    value = energy[energy['Country']==country].index
    energy.drop(labels=value, axis=0, inplace=True)

In [32]:
#Rechecking missing values
energy.isnull().sum()

Country                          0
Energy_type                      0
Year                             0
Energy_consumption               0
Energy_production                0
GDP                            128
Population                       0
Energy_intensity_per_capita      0
Energy_intensity_by_GDP          0
CO2_emission                     0
dtype: int64

In [33]:
energy.shape

(35922, 10)

Dropped remaining missing values

In [34]:
energy = energy[pd.notnull(energy['GDP'])]

In [35]:
energy.isnull().sum()

Country                        0
Energy_type                    0
Year                           0
Energy_consumption             0
Energy_production              0
GDP                            0
Population                     0
Energy_intensity_per_capita    0
Energy_intensity_by_GDP        0
CO2_emission                   0
dtype: int64

In [36]:
co2= energy.copy()

In [37]:
co2

,Country,Energy_type,Year,Energy_consumption,Energy_production,GDP,Population,Energy_intensity_per_capita,Energy_intensity_by_GDP,CO2_emission
11088,World,all_energy_types,1988,345.560876,347.412863,42106.595403,4.927545e+06,70.128405,8.206811,21163.840556
11089,World,coal,1988,96.873178,98.484482,42106.595403,4.927545e+06,70.128405,8.206811,8930.924825
11090,World,natural_gas,1988,71.010048,71.852938,42106.595403,4.927545e+06,70.128405,8.206811,3571.676242
11091,World,petroleum_n_other_liquids,1988,133.445814,132.485303,42106.595403,4.927545e+06,70.128405,8.206811,8661.239490
11092,World,nuclear,1988,19.226897,19.226897,42106.595403,4.927545e+06,70.128405,8.206811,0.000000
...,...,...,...,...,...,...,...,...,...,...
55435,Zimbabwe,coal,2019,0.045064,0.075963,37.620400,1.465420e+04,11.508701,4.482962,4.586869
55436,Zimbabwe,natural_gas,2019,0.000000,0.000000,37.620400,1.465420e+04,11.508701,4.482962,0.000000
55437,Zimbabwe,petroleum_n_other_liquids,2019,0.055498,0.000000,37.620400,1.465420e+04,11.508701,4.482962,4.377890
55438,Zimbabwe,nuclear,2019,0.000000,0.000000,37.620400,1.465420e+04,11.508701,4.482962,0.000000


### Creating Continents for Simpler Analysis

Create a list of all unique countries

In [38]:
countries= co2['Country'].unique()
countries= countries.tolist() 

If you do not have it, pip install pycountry-convert to help group countries by thei corresponding or surrounding continent

In [39]:
# !pip install pycountry-convert

Remove the countries pycountry cannot place in continent

In [40]:

otherC= ['World','Burma','Congo-Brazzaville', 'Congo-Kinshasa', 'Palestinian Territories','The Bahamas','Reunion',  'Kosovo', 'Timor-Leste', "Côte d’Ivoire", 'Gambia, The', 'Saint Vincent/Grenadines']

for i in otherC:
    countries.remove(i)

Convert to Continent 

In [41]:
import pycountry_convert as pc

def country_to_continent(country_name):
    country_alpha2 = pc.country_name_to_country_alpha2(country_name)
    country_continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
    country_continent_name = pc.convert_continent_code_to_continent_name(country_continent_code)
    return country_continent_name

Continent= {}
Asia= []
Europe= []
Africa= []
NAmerica= []
SAmerica= []
Oceania=  []
for item in countries:
    cont= country_to_continent(item)
    if cont == 'Asia':
        Asia.append(item)
    elif cont == 'Europe':
        Europe.append(item)
    elif cont == 'North America':
        NAmerica.append(item)
    elif cont == 'South America':
        SAmerica.append(item)
    elif cont == 'Oceania':
        Oceania.append(item)
    elif cont == 'Africa':
        Africa.append(item)
    else:
        pass


Manually add remaining countries to its designated continent

In [42]:
AC= ['Burma','Palestinian Territories', 'Timor-Leste']
AFC= ['Congo-Brazzaville', 'Congo-Kinshasa', 'Reunion', "Côte d’Ivoire", 'Gambia, The']
NC= ['The Bahamas', 'Saint Vincent/Grenadines']
for item in AC:
    Asia.append(item)
for item in AFC:
    Africa.append(item)
for item in NC:
    NAmerica.append(item)
Europe.append('Kosovo')

Build Continent Dictionary

In [43]:
Continent['Asia'] = Asia
Continent['Europe']= Europe
Continent['Africa'] = Africa
Continent['North America']= NAmerica
Continent['South America'] = SAmerica
Continent['Oceania']= Oceania


Add contents of the Continent Dictionary to co2 dataframe

In [44]:

def return_continent(row) : 
    try : 
        for continent in Continent.keys() : 
            if row['Country'] in Continent[continent]: 
                return continent
    except : 
        return "Others"
co2['Continent'] = co2.apply(return_continent, axis = 1)

For World, the continents comes out as null. We replaced these nulls with Other

In [45]:
co2['Continent'].isnull().sum()
co2["Continent"].fillna("Other", inplace = True)

Make csv with cleaned data using Pandas

In [46]:
co2.to_csv('cleaned_energy.csv')